## SupraMolSim Analysis workflows focus on exploting its core methods (showcased in general workflows) within parameter sweeps to explore the imaging space!


### In order to do so, we will use the Experiment module, desinged to ease the navigation of SupraMolSim functionallity at any point in the workflow.
The Experiment itself allows you to initialise a demo experiment


In [1]:
from supramolsim.analysis import _plots, sweep
import numpy as np

In [ ]:
structures = ["7R5K", ]
probes=["Mock_linker", ]
probe_parameters_vectors = dict(
    target_info = [dict(type="Sequence", value="ELAVGSL"),],
    labelling_efficiency = np.linspace(0.5,1,5),
    distance_to_epitope = np.linspace(50,200,1)
)
modalities = dict(
    STED=dict(
        nframes=2,
        exp_time=0.005

    ), # change nframes
    Confocal=dict(
        nframes=2,
        exp_time=0.005
    ),
)
particle_defects = dict(
    eps1 = np.linspace(300,300,1),
    eps2 = np.linspace(600,600,1),
    defect = np.linspace(0,0.5,1)
)
sweep_repetitions = 3

field_parameters_vector = dict(
    random_orientations = [False,],
)


probe_parameters = sweep.create_param_combinations(**probe_parameters_vectors)
p_Defects = sweep.create_param_combinations(**particle_defects)
p_sample = sweep.create_param_combinations(**field_parameters_vector)

# Create references

In [ ]:
ref_probe_parameters = dict(
    target_info = dict(type="Sequence", value="ELAVGSL"),
    labelling_efficiency = 1,
    distance_to_epitope = 0
)
ref_vsample, ref_params = sweep.generate_global_reference_sample(structure=structures[0], probe="Mock_linker", probe_parameters=ref_probe_parameters)
ref_image, ref_image_pars = sweep.generate_global_reference_modality(reference_vsample=ref_vsample, reference_vsample_params=ref_params)


# sweep over parameters

In [ ]:
myexperiment, vsmpl_output, vsampl_pars = sweep.sweep_vasmples(
    structures=structures,
    probes=probes, 
    probe_parameters=probe_parameters, 
    particle_defects=p_Defects,
    virtual_samples=p_sample,
    repetitions=sweep_repetitions)

In [ ]:
myexperiment, mod_outputs, mod_params, mod_pixelsizes = sweep.sweep_modalities(myexperiment, vsmpl_output, vsampl_pars, modalities="all")


# explore outputs

In [ ]:
import matplotlib.pyplot as plt
param_id = list(mod_params.keys())[3]
repetition = 0
frame = 0

plt.imshow(mod_outputs[param_id][repetition][frame])
print(mod_params[param_id])

# analyse sweep outputs

In [ ]:
sweep_analyse_parameters = dict()
ref_pixelsize = ref_image_pars["ref_pixelsize"]

for mod_name, pixelsize in mod_pixelsizes.items():
    sweep_analyse_parameters[mod_name] = {}
    sweep_analyse_parameters[mod_name]["metric"] = "ssim"
    sweep_analyse_parameters[mod_name]["modality_pixelsize"] = pixelsize
    sweep_analyse_parameters[mod_name]["ref_pixelsize"] = ref_pixelsize
    sweep_analyse_parameters[mod_name]["force_match"] = True
sweep_analyse_parameters

In [ ]:
measurements, inputs = sweep.analyse_image_sweep(mod_outputs, mod_params, ref_image, sweep_analyse_parameters)
dframe, combined = sweep.measurements_dataframe(measurements, probe_parameters, p_Defects, p_sample, mod_params)

# Plot measurements

In [ ]:
df_categories, titles = sweep.pivot_dataframes_byCategory(combined, "modality_name", "labelling_efficiency", "distance_to_epitope")
_plots.sns_heatmap_pivots(df_categories, titles, annotations=True)